In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import io
from scipy.stats import ttest_ind
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import OneHotEncoder
import chardet


# Step 1: Ask at least two questions
# Predictive Question: Given a parent’s level of education, how likely are they going to have exam prep?
# Exploratory Question: Whether students who had exam prep have higher scores in our sample data?

# Step 2: Collect the data
# Provide the data source
data_source = "https://www.kaggle.com/datasets/spscientist/students-performance-in-exams. html5lib "
# How was the data collected?
data_collection_source = "http://roycekimmons.com/tools/generated_data/exams. html5lib "
# This is a fictional dataset and should only be used for data science training purposes.

# Reads data file from Google Drive
file_id = "1boOd8BVffLE914tY4n43Msyrvpx8LygS"
url = f"https://drive.google.com/uc?id={file_id}"
download = requests.get(url).content


# Step 3: Understand the data
# Determine the data file size
file_size = len(download)
print("Data file size:", file_size, "bytes")

# Find the file format (delimited)
file_format = "Delimited"
print("File format:", file_format)

# Determine the file encoding (using chardet)
file_encoding = detected_encoding, ut
print("File encoding:", file_encoding)
# Read the data using the detected encoding
df = pd.read_csv(io.StringIO(download.decode(detected_encoding)))

# What is the granularity of the data? (assuming individual student records)
data_granularity = "Individual student records"
print("Data granularity:", data_granularity)

# Data Quality Check: Are there any outliers? Are there any missing or invalid entries?
# Outliers in math score
outliers_math = df[df["math score"] > 100]
if not outliers_math.empty:
    print("There are outliers in the math score column.")
    # Data cleaning step: Replace outliers with the maximum valid score of 100
    df.loc[df["math score"] > 100, "math score"] = 100

# Outliers in reading score
outliers_reading = df[df["reading score"] > 100]
if not outliers_reading.empty:
    print("There are outliers in the reading score column.")
    # Data cleaning step: Replace outliers with the maximum valid score of 100
    df.loc[df["reading score"] > 100, "reading score"] = 100

# Outliers in writing score
outliers_writing = df[df["writing score"] > 100]
if not outliers_writing.empty:
    print("There are outliers in the writing score column.")
    # Data cleaning step: Replace outliers with the maximum valid score of 100
    df.loc[df["writing score"] > 100, "writing score"] = 100

# Missing or invalid entries
missing_values = df.isnull().sum()
if missing_values.any():
    print("There are missing or invalid entries in the data set.")
    # Data cleaning step: Handle missing values in "test preparation course" column with the most frequent value (mode)
    mode_prep_course = df["test preparation course"].mode().iloc[0]
    df["test preparation course"].fillna(mode_prep_course, inplace=True)

# Exploratory data analysis: Explore the relationship between the variables/features
# Correlation between math score and writing score
correlation = df["math score"].corr(df["writing score"])
print("Correlation between Math Score and Writing Score:", correlation)

# Visualization: Scatter plot of math score and writing score
plt.scatter(df["math score"], df["writing score"])
plt.title("Math Score vs Writing Score")
plt.xlabel("Math Score")
plt.ylabel("Writing Score")
plt.show()

# Bar plot of exam preparation and average scores
avg_scores = df.groupby("test preparation course")[["math score", "reading score", "writing score"]].mean()
avg_scores.plot(kind="bar")
plt.title("Average Scores by Exam Preparation")
plt.xlabel("Exam Preparation")
plt.ylabel("Average Score")
plt.show()

# Inferential analysis: Compare scores between students who completed the test preparation course and those who did not
completed_prep = df[df["test preparation course"] == "completed"]["math score"]
not_completed_prep = df[df["test preparation course"] == "none"]["math score"]

# Perform t-test
t_stat, p_value = ttest_ind(completed_prep, not_completed_prep)

# Determine the significance level
significance_level = 0.05

# Make a decision based on the p-value
if p_value < significance_level:
    print("Students who completed the test preparation course have a significantly higher score.")
else:
    print("There is not enough evidence to conclude a significant difference in scores.")

# Exploratory Question: How many people scored higher than 85 in math?
num_high_scores = len(df[df["math score"] > 85])
print("Number of people scoring higher than 85 in math:", num_high_scores)

# Inferential analysis for reading and writing scores: Compare scores between students who completed the test preparation course and those who did not
completed_prep_reading = df[df["test preparation course"] == "completed"]["reading score"]
not_completed_prep_reading = df[df["test preparation course"] == "none"]["reading score"]

completed_prep_writing = df[df["test preparation course"] == "completed"]["writing score"]
not_completed_prep_writing = df[df["test preparation course"] == "none"]["writing score"]

# Perform t-tests
t_stat_reading, p_value_reading = ttest_ind(completed_prep_reading, not_completed_prep_reading)
t_stat_writing, p_value_writing = ttest_ind(completed_prep_writing, not_completed_prep_writing)

# Make decisions based on the p-values
if p_value_reading < significance_level:
    print("Students who completed the test preparation course have a significantly higher reading score.")
else:
    print("There is not enough evidence to conclude a significant difference in reading scores.")

if p_value_writing < significance_level:
    print("Students who completed the test preparation course have a significantly higher writing score.")
else:
    print("There is not enough evidence to conclude a significant difference in writing scores.")



# Step 4: Prepare the data for machine learning
# Perform OneHotEncoding for the "gender", "race/ethnicity", "parental level of education", and "lunch" features
categorical_features = df[["gender", "race/ethnicity", "parental level of education", "lunch"]]

# Initialize OneHotEncoder
ohe = OneHotEncoder(sparse=False)  # Set `sparse` to False for easier manipulation

# Fit and transform the categorical features
encoded_features = ohe.fit_transform(categorical_features)

# Get the feature names for the encoded features
unique_values = [col + "_" + str(val) for col, vals in zip(categorical_features.columns, ohe.categories_) for val in vals]
df_encoded_features = pd.DataFrame(encoded_features, columns=unique_values)

# Concatenate the encoded features with the original DataFrame
df_encoded = pd.concat([df, df_encoded_features], axis=1)

# Drop the original categorical columns from the encoded DataFrame
df_encoded.drop(columns=["gender", "race/ethnicity", "parental level of education", "lunch"], inplace=True)

# Define the feature set X and target variable y
X = df_encoded.drop(["math score", "reading score", "writing score", "test preparation course"], axis=1)
# Map 'none' to 0 and 'completed' to 1 in the target variable
y = df_encoded["test preparation course"].map({'none': 0, 'completed': 1})

# Step 5: Build a machine learning model (Logistic Regression)
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the model to the training data
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Step 6: Evaluate the model
# Calculate accuracy and mean squared error
accuracy = accuracy_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

# Display Logistic Regression Model Evaluation
print("\nLogistic Regression Model Evaluation:")
print("Accuracy:", accuracy)
print("Mean Squared Error (MSE):", mse)


# Step 7: Share the insights
# Limitations of the analysis
print("\nLimitations:")
print("- The dataset is fictional and may not fully represent real-world data, which could impact the generalizability of the results.")
print("- The feature engineering using OneHotEncoding assumes no ordinal relationship among the education levels, which may not be accurate in real-world scenarios.")
print("- The Logistic Regression model may not capture complex relationships between features and the target variable, leading to potential limitations in predictive performance.")

# Interesting findings
print("\nInteresting Findings:")
print("- The scatter plot shows a positive correlation between math score and writing score. Higher math scores tend to correspond to higher writing scores.")
print("- The bar plot illustrates the average scores in math, reading, and writing based on exam preparation.")
print("- Students who completed the test preparation course have a significantly higher score (t-test).")
print(f"- The number of people scoring higher than 85 in math is: {num_high_scores}")
print("- Students who completed the test preparation course have a significantly higher reading score (t-test).")
print("- Students who completed the test preparation course have a significantly higher writing score (t-test).")



Data file size: 72036 bytes
File format: Delimited


NameError: ignored